In [ ]:
pip install -q replicate

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 48.0/48.0 kB 2.7 MB/s eta 0:00:00


In [ ]:
import time
import replicate
import pandas as pd

In [ ]:
df = pd.read_csv('/content/WELFake_Dataset.csv', nrows=500, on_bad_lines='skip')

In [ ]:
# Zero shot
def create_prompt(article_text):
    prompt = f"""
    Now classify this article:
    Article: "{article_text}"
    Label:
    """
    return prompt


In [ ]:
# zero shots
def create_prompt(article_text):
    prompt = f"""
    Classify the given article as either "real" or "fake."
    Article: "{article_text}"
    Label:
    """
    return prompt


In [ ]:
client = replicate.Client(api_token="")

In [ ]:
predictions = []

# Iterate over the 500 examples
for index, row in df.iterrows():
    article = row['text']  # assuming the text column is named 'text'
    prompt = create_prompt(article)

    # Call the model for prediction
    result = client.run(
        "ibm-granite/granite-3.0-8b-instruct",
        input={
        "top_k": 10,
        "prompt": prompt,
        "max_tokens": 10,
        "min_tokens": 1,
        "temperature": 0,
        "system_prompt": "You are an expert in fake news detection and your job is to classify articles into fake or real news. Only answer with 'Real' or 'Fake'",
        "repetition_penalty": 1,
        "presence_penalty": 0,
        "frequency_penalty": 0
    },
    )

    # Assuming the result is a string with 'Real' or 'Fake' as output
    predictions.append(result[-1].strip())  # Clean the result (strip spaces)

    # Introduce a delay of 1 second between requests
    time.sleep(0.5)
      # You can adjust the sleep time (in seconds) as needed

print(predictions[:5])  # Print first 5 predictions to check


['Fake', 'Fake', 'Fake', 'Real', 'Real']


In [ ]:
predictions

['Fake',
 'Fake',
 'Fake',
 'Real',
 'Real',
 'Fake',
 'Real',
 'confusion, but it seems',
 'Real',
 'Real',
 'Real',
 'Fake',
 'Real',
 'Fake',
 'Fake',
 'Real',
 'Real',
 'Real',
 'Fake',
 'Real',
 'Fake',
 'Fake',
 'Fake',
 '',
 'Real',
 'Real',
 'Real',
 'Real',
 'Real',
 'Fake',
 'Real',
 'it',
 'Fake',
 '',
 'Real',
 'Real',
 'a fake tweet from a',
 'Fake',
 'Fake',
 'Real',
 ', not',
 'Fake',
 'Fake',
 'Fake',
 'Real',
 'Real',
 '',
 'Real',
 '',
 'Real',
 'current state',
 'Real',
 'Real',
 'Fake',
 '',
 '',
 'Real',
 'Fake',
 'Real',
 'Real',
 'Real',
 'Fake',
 'Fake',
 'Real',
 'Fake',
 'Fake',
 'Real',
 'Real',
 'Fake',
 'Real',
 'Fake',
 'Real',
 'Real',
 'Real',
 'Fake',
 'Fake',
 'Real',
 'Fake',
 'Real',
 'Fake',
 'Real',
 'Real',
 'Real',
 'Real',
 'Fake',
 'Real',
 '',
 'Real',
 '',
 'Real',
 'Real',
 'Real',
 'Real',
 'a satirical piece from',
 'Real',
 'Real',
 'Real',
 'Fake',
 'likely a piece of propag',
 'Fake',
 'Real',
 'because it',
 '',
 'a political statement

In [ ]:
len(predictions)

500

In [ ]:
filtered_labels = ['Real' if 'Real' in label else 'Fake' if 'Fake' in label else 'Fake' for label in predictions]

len(filtered_labels)


500

In [ ]:
from sklearn.metrics import accuracy_score

# Assuming 'label' column contains the true labels
true_labels = df['label'].replace({1: 'Real', 0: 'Fake'}).values  # Replace with the actual column name for true labels
#true_labels = true_labels[:500]
# Calculate accuracy
accuracy = accuracy_score(true_labels, filtered_labels)
print(f"Accuracy: {accuracy * 100:.2f}%")


Accuracy: 29.00%


In [ ]:
full_predictions = ['Real' if 'Real' in label else 'Fake' if 'Fake' in label else None for label in predictions]


In [ ]:
df = pd.DataFrame({'True Labels': true_labels, 'Predictions': full_predictions})

# Remove rows where any column is None (or NaN)
df_cleaned = df.dropna()

In [ ]:
accuracy = accuracy_score(df_cleaned['True Labels'], df_cleaned['Predictions'])
print(f"Accuracy: {accuracy * 100:.2f}%")

Accuracy: 27.59%
